In [1]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score, KFold
from xgboost import DMatrix


### Input Data - Cleaned Dataset All

In [2]:

# df_TagDesc = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\CSV\Not for Processing\TagDesc.csv')

# df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
# df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# # Concatenate (union) the dataframes
# df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

df_All = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Merge Data\merged_data.csv')

print(df_All.head())


                  Date  425_pct_Al  C4_pct_Eth  C4_pct_H2O  \
0  2012-05-16 16:00:00    6.307630    2.508420    21.96340   
1  2012-05-16 18:00:00    6.308335    2.511135    21.96865   
2  2012-05-16 19:00:00    6.309390    2.515210    21.97655   
3  2012-05-16 20:00:00    6.310090    2.517925    21.98185   
4  2012-05-16 21:00:00    6.310790    2.520640    21.98715   

   HydWtr_pct_Ammonia  HydWtr_Na2O    Butanol   Decanol   DI55102   DI55152  \
0            0.908765     2.712180  54.858300  0.722511  0.970496  0.925255   
1            0.907643     2.805415  54.858300  0.722511  0.970496  0.925255   
2            0.905959     2.945270  54.858300  0.722511  0.970496  0.925255   
3            0.904837     3.038505  51.190050  0.733904  0.970653  0.923773   
4            0.903715     3.131740  48.744567  0.741499  0.970307  0.924103   

   ...      FC55569    FC55576  FFC55555    LC55557    LC90366    LC90368  \
0  ...  6918.110000  334.94800  0.751501  65.873400  61.011100  51.234400  

In [3]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [4]:
# # # List of columns to exclude from the XGboost feature selection results
# exclude_columns = [
#                     'TI40050',
#                     'TC55555',
#                     'LC55568',
#                     'LC55557',
#                     'Al2O3',
#                     'LC55553',
#                     'M_Value',
#                     'FFC55553',
#                     'DI55580',
#                     'Al2O3',
#                     'C4_pct_Hex',
#                     'LC52572',

# #                  'Date',
# #                  'C4_pct_Hex', 'HydWtr_Na2O',
# #                  'TC55555'  
#                    ]

# # # Create a new DataFrame without the excluded columnsd
# df_All = df_All.drop(columns=exclude_columns)

In [5]:
df_All.columns

Index(['Date', '425_pct_Al', 'C4_pct_Eth', 'C4_pct_H2O', 'HydWtr_pct_Ammonia',
       'HydWtr_Na2O', 'Butanol', 'Decanol', 'DI55102', 'DI55152', 'FC55003',
       'FC55009', 'FC55552', 'FC55569', 'FC55576', 'FFC55555', 'LC55557',
       'LC90366', 'LC90368', 'PI55004', 'PI55020', 'TC55552', 'TI55021'],
      dtype='object')

In [6]:
# df_All = df_All[df_All['Date'] > '2020-06-15 00:00:00']

In [7]:
# df_All.drop('Date', axis=1, inplace=True)




In [8]:
print(df_All.describe())

         425_pct_Al    C4_pct_Eth    C4_pct_H2O  HydWtr_pct_Ammonia  \
count  47882.000000  47882.000000  47882.000000        47882.000000   
mean       6.145878      1.234033     21.182765            0.968711   
std        0.257346      0.720766      2.741406            0.148502   
min        5.171905      0.086948      8.149630            0.389479   
25%        6.005205      0.631261     18.836025            0.877101   
50%        6.139890      1.119195     21.446875            0.940564   
75%        6.286790      1.698079     23.306850            1.031051   
max        7.134450      9.466675     34.572900            1.603285   

        HydWtr_Na2O       Butanol       Decanol       DI55102       DI55152  \
count  47882.000000  47882.000000  47882.000000  47882.000000  47882.000000   
mean       0.812908      9.835905      3.061965      0.944754      0.933222   
std        0.701055      9.743974      1.523367      0.052565      0.029663   
min        0.002854      0.361524      0.087

In [9]:
# # Assuming your data is in filtered_df and you want to predict 'target_column_name'
# df = df_All.drop(['Butanol', 'FC55009', 'PI55004', 'PI55020','TI55021'], axis=1)  # Replace 'target_column_name' with your target column's name
# # y = df_All['Decanol']

In [10]:
# # Keep only rows with specified IDs
# feature_names = [
# '425_pct_Al',
# 'C4_pct_Eth',
# 'C4_pct_H2O',
# 'HydWtr_pct_Ammonia',
# 'HydWtr_Na2O',
# 'DI55102',
# 'DI55152',
# 'FC55003',
# 'FC55009',
# 'FC55552',
# 'FC55569',
# 'FC55576',
# 'FFC55555',
# 'LC90366',
# 'LC90368',
# 'PI55004',
# 'PI55020',
# 'TC55552',
# 'TI55021'
# ]
    

In [11]:
import pickle

# Load the trained XGBoost models
with open(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C10\5 Preprocessing - Slecected Features\Updated Model\C10_xgb_model.sav', 'rb') as model_file:
    c10_model = pickle.load(model_file)
with open(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C4\5 Preprocessing - Slecected Features\Updated Model\C4_xgb_model.sav', 'rb') as model_file:
    c4_model = pickle.load(model_file)

# Initialize the new columns in the original df
df_All['FFC55555_adj'] = df_All['FFC55555']
df_All['Decanol_adj'] = None  # Using None as a placeholder
df_All['Butanol_adj'] = None  # Using None as a placeholder

def predict_value(model, row, feature_names):
    # Ensure 'row' is prepared correctly for your model
    data_for_prediction = row[feature_names].values.reshape(1, -1)
    prediction = model.predict(data_for_prediction)[0]
    return prediction

# Assume 'decanol_feature_names' and 'butanol_feature_names' are defined
# You should define these lists based on the features your models expect
decanol_feature_names = ['425_pct_Al', 'C4_pct_Eth', 'C4_pct_H2O', 'HydWtr_pct_Ammonia',
        'HydWtr_Na2O', 'DI55102', 'DI55152', 'FC55003', 'FC55009', 'FC55552', 'FC55569', 
        'FC55576', 'FFC55555', 'LC90366', 'LC90368', 'PI55004', 'PI55020', 'TC55552', 
        'TI55021']  
butanol_feature_names = ['425_pct_Al', 'C4_pct_Eth', 'C4_pct_H2O', 'HydWtr_pct_Ammonia',
       'HydWtr_Na2O', 'DI55102', 'DI55152', 'FC55003', 'FC55552',
       'FC55569', 'FC55576', 'FFC55555', 'LC55557', 'LC90366', 'LC90368',
       'TC55552']

# Your prediction and adjustment loop
for index, row in df_All.iterrows():
    # Adjust row for prediction if necessary
    row_for_prediction = row.copy()
    row_for_prediction['FFC55555'] = row_for_prediction['FFC55555_adj']
    
    # Process record through model C10 for Decanol
    df_All.at[index, 'Decanol_adj'] = predict_value(c10_model, row_for_prediction, decanol_feature_names)
    
    # Process record through model C4 for Butanol
    df_All.at[index, 'Butanol_adj'] = predict_value(c4_model, row_for_prediction, butanol_feature_names)

    # Process record through Optimization Logic
    while True:
        decanol = df_All.at[index, 'Decanol_adj']
        butanol = df_All.at[index, 'Butanol_adj']
        ffc_adjusted = df_All.at[index, 'FFC55555_adj']

        if decanol < 5:
            if ffc_adjusted >= 0.67: # Can be adjusted to the lowest rC4 desired
                df_All.at[index, 'FFC55555_adj'] -= 0.01
            else:
                break  # Next Record
        else:
            while decanol >= 5:
                if df_All.at[index, 'FFC55555'] >= ffc_adjusted:
                    break  # Next Record
                else:
                    if ffc_adjusted <= 1.19: # Can be adjusted to the highest rC4 desired
                        df_All.at[index, 'FFC55555_adj'] += 0.01
                    else:
                        break
            break

        # Update predictions after adjustment
        row_for_prediction['FFC55555'] = df_All.at[index, 'FFC55555_adj']
        df_All.at[index, 'Decanol_adj'] = predict_value(c10_model, row_for_prediction, decanol_feature_names)
        df_All.at[index, 'Butanol_adj'] = predict_value(c4_model, row_for_prediction, butanol_feature_names)


In [12]:
df_All.head()

,Date,425_pct_Al,C4_pct_Eth,C4_pct_H2O,HydWtr_pct_Ammonia,HydWtr_Na2O,Butanol,Decanol,DI55102,DI55152,FC55003,FC55009,FC55552,FC55569,FC55576,FFC55555,LC55557,LC90366,LC90368,PI55004,PI55020,TC55552,TI55021,FFC55555_adj,Decanol_adj,Butanol_adj
0,2012-05-16 16:00:00,6.307630,2.508420,21.96340,0.908765,2.712180,54.858300,0.722511,0.970496,0.925255,4919.29,1528.010000,41564.100000,6918.110000,334.94800,0.751501,65.873400,61.011100,51.234400,1.306310,-0.607993,177.188,212.861000,0.661501,1.202501,55.402496
1,2012-05-16 18:00:00,6.308335,2.511135,21.96865,0.907643,2.805415,54.858300,0.722511,0.970496,0.925255,4919.29,1528.010000,41564.100000,6918.110000,334.94800,0.751501,65.873400,61.011100,51.234400,1.306310,-0.607993,177.188,212.861000,0.661501,1.201722,54.622807
2,2012-05-16 19:00:00,6.309390,2.515210,21.97655,0.905959,2.945270,54.858300,0.722511,0.970496,0.925255,4919.29,1528.010000,41564.100000,6918.110000,334.94800,0.751501,65.873400,61.011100,51.234400,1.306310,-0.607993,177.188,212.861000,0.661501,1.196464,54.384926
3,2012-05-16 20:00:00,6.310090,2.517925,21.98185,0.904837,3.038505,51.190050,0.733904,0.970653,0.923773,5268.62,1516.963333,41524.066667,6916.486667,332.47200,0.750505,65.957600,56.694400,50.868167,1.216603,-0.626386,177.417,212.822333,0.660505,1.137032,49.17197
4,2012-05-16 21:00:00,6.310790,2.520640,21.98715,0.903715,3.131740,48.744567,0.741499,0.970307,0.924103,5349.32,1515.782500,41500.275000,6918.470000,338.63375,0.750325,65.966425,55.181525,50.740225,1.206718,-0.632433,177.459,212.799750,0.660325,1.137636,49.436687


In [13]:
df_All.to_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_66.csv', index=False)